# Calculate robusteness from pi control runs

In [399]:
# Some definitions

# directory of data
#fdir = '~/Google Drive/professional/research/FARALLON_INSTITUTE_PROJECTS/2020 NOAA MAPP/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'
fdir = '/Volumes/GoogleDrive/My Drive/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'

In [400]:
# Modules
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore') #filter some warning messages
import seaborn as sns

In [403]:
def num_coevents(y1, y2, pct1, pct2, runs='picontrol'):
    # 
    fin1 = fdir+'anomaly ts/'+runs+'_sst_anom.nc'
    fin2 = fdir+'anomaly ts/'+runs+'_sm_anom.nc'
    modsst = xr.open_dataset(fin1)
    modsst.close()
    modsm2 = xr.open_dataset(fin2)
    modsm2.close()
    
    models = modsm2.model

    # selec periood
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))
    
    nev = list()
    
    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values
    
        # calculate threshold
        mhw_thr = np.nanpercentile(tmp1,pct1)
        drg_thr = np.nanpercentile(tmp2,pct2)
    
        a1 = tmp1>=mhw_thr
        a2 = tmp2<=drg_thr
        
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]
    
        nev.append(len(tmp)) #/(y2-y1+1)) #freq.
        
    return nev  # sum(nev)

In [404]:
ny = 30
prc1 = 90
prc2 = 10

pinev=list()
for i in range(int(502/ny)):
    nev = num_coevents(i*ny, (i+1)*ny-1, prc1 , prc2)
    pinev.append(nev)
pinev,np.mean(pinev) 

([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1],
  [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0],
  [0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
  [0, 0, 0, 1, 1, 0,

In [405]:
#model by model

In [406]:
model_mean=np.mean(pinev,axis=0)
model_spread=np.std(pinev,axis=0)
model_mean

array([0.0625, 0.125 , 0.1875, 0.125 , 0.25  , 0.0625, 0.3125, 0.4375,
       0.1875, 0.    , 0.25  , 0.1875, 0.25  , 0.1875, 0.1875, 0.1875,
       0.0625, 0.125 , 0.25  , 0.1875, 0.25  , 0.25  ])

In [407]:
#16 30-yr periods MMEs

In [408]:
mme_mean=np.mean(pinev,axis=1)
mme_spread=np.std(pinev,axis=1)
#internal variability mean and spread
mme_mean,np.mean(mme_spread)

(array([0.        , 0.        , 0.09090909, 0.27272727, 0.27272727,
        0.40909091, 0.18181818, 0.27272727, 0.22727273, 0.        ,
        0.        , 0.22727273, 0.31818182, 0.27272727, 0.13636364,
        0.31818182]),
 0.32825512607124546)

In [409]:
#MME mean and spread

In [410]:
mme_model_mean=np.mean(mme_mean)
mme_model_spread=np.std(mme_mean)
mme_model_mean,mme_model_spread

(0.1875, 0.12944074715789225)

In [411]:
#cross model mean and spread

In [412]:
cross_model_mean=np.mean(model_mean)
cross_model_spread=np.std(model_mean)
cross_model_mean,cross_model_spread

(0.1875, 0.09422229518055114)

In [413]:
## Calculate thresholds for PI Control runs

In [414]:
def get_thre(y1,y2,prc1,prc2):
    # open data: anomalies (without trend)

    modsst = xr.open_dataset(fdir+'anomaly ts/picontrol_sst_anom.nc')
    modsst.close()
    modsm2 = xr.open_dataset(fdir+'anomaly ts/picontrol_sm_anom.nc')
    modsm2.close()

    # select period 
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))
    mhw_thr = list()
    drg_thr = list()
    models = list()

    # Calculate thresholds for each model
    for ix,i in enumerate(modsm2.model.values):
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values
        
        mhw_thr.append(np.nanpercentile(tmp1,prc1))
        drg_thr.append(np.nanpercentile(tmp2,prc2))

        models.append(i)
    
    return mhw_thr, drg_thr , models

In [419]:
# calculate number of events for control, historical and future
# based on pi control thresholds

def freq_coevents(fdir, runs, mhw_thr, drg_thr, models, y1, y2):
    #pi control
    fi1 = fdir+'anomaly ts/'+runs+'_sst_anom.nc'
    fi2 = fdir+'anomaly ts/'+runs+'_sm_anom.nc'
    modsst = xr.open_dataset(fi1)
    modsst.close()
    modsm2 = xr.open_dataset(fi2)
    modsm2.close()
    models=modsm2.model

    # select period
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))

    nev = list()

    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values

        a1 = tmp1>=mhw_thr[ix]
        a2 = tmp2<=drg_thr[ix]
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]

        nev.append(len(tmp))
    
    freq = np.array(nev) #/(y2-y1+1)
    
    return nev, np.round(np.nanmean(nev),4), np.round(np.nanstd(nev),4) #mean no. of events
#freq, np.round(np.nanmean(freq),4), np.round(np.nanstd(freq),4) #mean frequency


In [420]:
def freq_difruns(ny, y0,y,prc1, prc2):
    
    # calculate threshold in pi control
    #y2=2014
    #y1=y2-ny+1
    mhw_thr, drg_thr, models = get_thre(y0,y,prc1,prc2)

    print ('Percentiles: '+str(prc1)+'/'+str(prc2))
    # pi control
    print('piControl: '+str(y0)+'-'+str(y))
    #y2=2014
    #y1=y2-ny+1
    freq0, mfrq, sfrq = freq_coevents(fdir,'picontrol',mhw_thr,drg_thr, models,y0,y)
    print(mfrq,sfrq)

    # historical 1
    y1 = 1900
    y2 = y1+ny-1
    print('\nhistorical: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'historical',mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

    # historical 2
    y2=2014
    y1=y2-ny+1
    print('\nhistorical 2: '+str(y1)+'-'+str(y2))
    freq1, mfrq, sfrq = freq_coevents(fdir,'historical',mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

    # future
    y2=2099
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq2, mfrq, sfrq = freq_coevents(fdir,'ssp585',mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)
    return freq0,freq1,freq2

In [421]:
# Frequency of events - detrended data, using pi control threshols
## percentiles 90-10

In [422]:

print ('\nPercentiles: '+str(prc1)+'/'+str(prc2))
print('\nProbable # events in '+str(ny)+' years'+': '+str(ny*0.01))

funev=list()
hinev=list()
test=list()
for i in range(int(502/ny)):
    pi,hi,fu = freq_difruns(ny,i*ny, (i+1)*ny-1, prc1, prc2)
    test.append(pi)
    funev.append(fu)
    hinev.append(hi)
np.mean(funev),np.mean(hinev),np.std(np.mean(hinev,axis=0))


Percentiles: 90/10

Probable # events in 30 years: 0.3
Percentiles: 90/10
piControl: 0-29
0.0 0.0

historical: 1900-1929
0.0 0.0

historical 2: 1985-2014
0.0 0.0

future: 2070-2099
0.0 0.0
Percentiles: 90/10
piControl: 30-59
0.0 0.0

historical: 1900-1929
0.0 0.0

historical 2: 1985-2014
0.0 0.0

future: 2070-2099
0.0 0.0
Percentiles: 90/10
piControl: 60-89
0.0909 0.2875

historical: 1900-1929
0.8182 1.0285

historical 2: 1985-2014
0.7273 0.8624

future: 2070-2099
0.5455 0.6556
Percentiles: 90/10
piControl: 90-119
0.2727 0.4454

historical: 1900-1929
0.4545 0.782

historical 2: 1985-2014
0.5 0.8394

future: 2070-2099
0.2273 0.4191
Percentiles: 90/10
piControl: 120-149
0.2727 0.4454

historical: 1900-1929
0.3182 0.8732

historical 2: 1985-2014
0.5455 1.0325

future: 2070-2099
0.4091 1.267
Percentiles: 90/10
piControl: 150-179
0.4091 0.4917

historical: 1900-1929
0.3182 0.5548

historical 2: 1985-2014
0.4545 0.6556

future: 2070-2099
0.2273 0.8492
Percentiles: 90/10
piControl: 180-209
0

(0.29829545454545453, 0.3778409090909091, 0.39189102689754196)

In [423]:
np.mean(test),np.mean(pinev)

(0.1875, 0.1875)

In [424]:
#model by model

In [425]:
fu_model_mean=np.mean(funev,axis=0)
fu_model_spread=np.std(funev,axis=0)
fu_model_mean

array([0.    , 0.    , 0.375 , 0.    , 0.25  , 0.    , 0.3125, 0.4375,
       0.6875, 0.0625, 1.1875, 1.    , 0.375 , 0.1875, 0.5   , 0.6875,
       0.0625, 0.    , 0.125 , 0.25  , 0.    , 0.0625])

In [426]:
np.mean(fu_model_mean),np.std(fu_model_mean)

(0.29829545454545453, 0.3306823063101677)

In [427]:
#mme mean and spread

In [428]:
fu_mme_mean=np.mean(funev,axis=1)
fu_mme_spread=np.std(funev,axis=1)
np.mean(fu_mme_spread)

0.5695171333166422

In [429]:
fu_mme_mean,np.mean(fu_mme_mean),np.std(fu_mme_mean)

(array([0.        , 0.        , 0.54545455, 0.22727273, 0.40909091,
        0.22727273, 0.63636364, 0.5       , 0.54545455, 0.        ,
        0.        , 0.40909091, 0.31818182, 0.18181818, 0.40909091,
        0.36363636]),
 0.29829545454545453,
 0.20889859609488337)

In [430]:
#16 MMEs diff and spread

In [431]:
diff=list()
zip_obj=zip(fu_mme_mean,mme_mean)
for i , j in zip_obj:
        diff.append(i-j)
diff

[0.0,
 0.0,
 0.4545454545454545,
 -0.04545454545454544,
 0.1363636363636364,
 -0.18181818181818185,
 0.45454545454545453,
 0.2272727272727273,
 0.3181818181818181,
 0.0,
 0.0,
 0.18181818181818185,
 0.0,
 -0.09090909090909088,
 0.27272727272727276,
 0.04545454545454547]

In [432]:
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(0.11079545454545454, 0.18391458306408537)

In [433]:
#model by model diff and spread

In [434]:
diff=list()
zip_obj=zip(fu_model_mean,model_mean)
for i , j in zip_obj:
        diff.append(i-j)
diff

[-0.0625,
 -0.125,
 0.1875,
 -0.125,
 0.0,
 -0.0625,
 0.0,
 0.0,
 0.5,
 0.0625,
 0.9375,
 0.8125,
 0.125,
 0.0,
 0.3125,
 0.5,
 0.0,
 -0.125,
 -0.125,
 0.0625,
 -0.25,
 -0.1875]

In [435]:
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(0.11079545454545454, 0.3084578451783422)